In [17]:
import os
import pandas as pd


In [18]:
import sys
import os

# Add the project directory to the PYTHONPATH
project_path = os.path.abspath(os.path.join(os.getcwd(), '../../..'))
if project_path not in sys.path:
    sys.path.append(project_path)

# Now you can import your custom module
from data_utils.data_processing import download_file, process_zip_file


In [19]:
# Define headers for CSV files without headers
DEFAULT_HEADERS = [
    "AÑO", "FECHA_DEF", "SEXO_NOMBRE", "EDAD_TIPO", "EDAD_CANT", "COD_COMUNA", "COMUNA", "NOMBRE_REGION",
    "DIAG1", "CAPITULO_DIAG1", "GLOSA_CAPITULO_DIAG1", "CODIGO_GRUPO_DIAG1", "GLOSA_GRUPO_DIAG1",
    "CODIGO_CATEGORIA_DIAG1", "GLOSA_CATEGORIA_DIAG1", "CODIGO_SUBCATEGORIA_DIAG1", "GLOSA_SUBCATEGORIA_DIAG1",
    "DIAG2", "CAPITULO_DIAG2", "GLOSA_CAPITULO_DIAG2", "CODIGO_GRUPO_DIAG2", "GLOSA_GRUPO_DIAG2",
    "CODIGO_CATEGORIA_DIAG2", "GLOSA_CATEGORIA_DIAG2", "CODIGO_SUBCATEGORIA_DIAG2", "GLOSA_SUBCATEGORIA_DIAG2",
    "LUGAR_DEFUNCION"
]


In [20]:
def get_data_paths():
    # Check if running in the GitHub Actions environment
    if 'GITHUB_ACTIONS' in os.environ:
        base_path = os.getcwd()
    else:
        # Assuming your script is in the 'scripts' directory
        base_path = os.path.abspath(os.path.join(os.getcwd(), '../../../data'))

    source_path = os.path.join(base_path, "source/minsal/deis")
    processed_path = os.path.join(base_path, "processed/minsal/deis")
    
    return source_path, processed_path

In [21]:
# Create directories if they don't exist
source_dir, processed_dir = get_data_paths()
os.makedirs(source_dir, exist_ok=True)
os.makedirs(processed_dir, exist_ok=True)



In [22]:
# List of file URLs and corresponding CSV filenames to extract
file_info = [
    ("https://repositoriodeis.minsal.cl/DatosAbiertos/VITALES/DEFUNCIONES_FUENTE_DEIS_2022_2024_25062024.zip", "DEFUNCIONES_FUENTE_DEIS_2022_2024_25062024.csv"),
    ("https://repositoriodeis.minsal.cl/DatosAbiertos/VITALES/DEFUNCIONES_FUENTE_DEIS_1990_2021_CIFRAS_OFICIALES.zip", "DEFUNCIONES_FUENTE_DEIS_1990_2021_CIFRAS_OFICIALES.csv")
]



In [23]:
header_option = 'infer' if '2022_2024' in "https://repositoriodeis.minsal.cl/DatosAbiertos/VITALES/DEFUNCIONES_FUENTE_DEIS_2022_2024_25062024.zip" else None

In [28]:
header_option

'infer'

In [42]:
names_option = None if header_option == 'infer' else DEFAULT_HEADERS

In [43]:
url = file_info[0][0]

In [44]:
extract_filename = file_info[0][1]

In [45]:
names_option = DEFAULT_HEADERS

In [47]:
url

'https://repositoriodeis.minsal.cl/DatosAbiertos/VITALES/DEFUNCIONES_FUENTE_DEIS_2022_2024_25062024.zip'

In [48]:
process_zip_file(url, extract_filename, source_dir, processed_dir, header=header_option, names=names_option)

DEFUNCIONES_FUENTE_DEIS_2022_2024_25062024.zip already exists. Skipping download.
Detected encoding for /tmp/DEFUNCIONES_FUENTE_DEIS_2022_2024_25062024.csv: ISO-8859-1
Successfully read file with encoding ISO-8859-1
Processed DEFUNCIONES_FUENTE_DEIS_2022_2024_25062024.csv and saved to /Users/ernestolaval/Documents/nodeJS/github/datos_abiertos/data/processed/minsal/deis/DEFUNCIONES_FUENTE_DEIS_2022_2024_25062024.parquet


In [49]:
# Process each zip file
for url, extract_filename in file_info:
    header_option = None if '2022_2024' in extract_filename else 'infer'
    names_option = None if header_option == 'infer' else DEFAULT_HEADERS
    process_zip_file(url, extract_filename, source_dir, processed_dir, header=header_option, names=names_option)



DEFUNCIONES_FUENTE_DEIS_2022_2024_25062024.zip already exists. Skipping download.
Detected encoding for /tmp/DEFUNCIONES_FUENTE_DEIS_2022_2024_25062024.csv: ISO-8859-1
Successfully read file with encoding ISO-8859-1
Processed DEFUNCIONES_FUENTE_DEIS_2022_2024_25062024.csv and saved to /Users/ernestolaval/Documents/nodeJS/github/datos_abiertos/data/processed/minsal/deis/DEFUNCIONES_FUENTE_DEIS_2022_2024_25062024.parquet
DEFUNCIONES_FUENTE_DEIS_1990_2021_CIFRAS_OFICIALES.zip already exists. Skipping download.
Detected encoding for /tmp/DEFUNCIONES_FUENTE_DEIS_1990_2021_CIFRAS_OFICIALES.csv: ISO-8859-1
Successfully read file with encoding ISO-8859-1
Processed DEFUNCIONES_FUENTE_DEIS_1990_2021_CIFRAS_OFICIALES.csv and saved to /Users/ernestolaval/Documents/nodeJS/github/datos_abiertos/data/processed/minsal/deis/DEFUNCIONES_FUENTE_DEIS_1990_2021_CIFRAS_OFICIALES.parquet


In [52]:
# Read and merge both files using the common structure
df1 = pd.read_parquet(os.path.join(processed_dir, "DEFUNCIONES_FUENTE_DEIS_2022_2024_25062024.parquet"))
df2 = pd.read_parquet(os.path.join(processed_dir, "DEFUNCIONES_FUENTE_DEIS_1990_2021_CIFRAS_OFICIALES.parquet"))

# Concatenate dataframes
df_combined = pd.concat([df1, df2], ignore_index=True)

# Filter records with AÑO >= 1997
df_filtered = df_combined[df_combined['AÑO'] >= 2003]

# Save combined and filtered dataframe
combined_parquet_path = os.path.join(processed_dir, "combined_defunciones_filtered.parquet")
df_filtered.to_parquet(combined_parquet_path)
print(f"Combined and filtered DataFrame saved to {combined_parquet_path}")

Combined and filtered DataFrame saved to /Users/ernestolaval/Documents/nodeJS/github/datos_abiertos/data/processed/minsal/deis/combined_defunciones_filtered.parquet
